## Asterix Cat 10 evaluator

Python notebook to analize SMR asterix cat 10 data


In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import json


def readFile(fileName):
    '''
    read asterix from json file and return two data structures
    '''

    with open(fileName, 'r') as f:
        data = [json.loads(line[:-2]) for line in f.readlines()]
    
    # f = open(fileName)
    # data = json.load(f)

    trackList = []
    track = {}
    trackIndices = {}
    index = 0

    for e in data[:]:
        try:
            trackNb = e['CAT010']['I161']['TrkNb']
        except:
            trackNb = ''
        if trackNb:
            # if str(trackNb) not in trackIndices:
            if trackNb not in trackIndices:
                trackIndices.update({trackNb: index})
                try:
                    plotLWO = e['CAT010']['I270']
                except:
                    plotLWO = ''
                try:
                    plotXY = e['CAT010']['I042']
                except:
                    plotXY = ''
                if plotLWO and plotXY:
                    track = {'track': trackNb,
                            'data': {'ToD': [e['CAT010']['I140']['ToD']],
                                     'X': [e['CAT010']['I042']['X']],
                                     'Y': [e['CAT010']['I042']['Y']],
                                     'Width': [e['CAT010']['I270']['Width']],
                                     'Length': [e['CAT010']['I270']['Length']],
                                     'Ori': [e['CAT010']['I270']['Ori']]
                                    }
                            }
                elif plotXY and not plotLWO:
                    track = {'track': trackNb,
                            'data': {'ToD': [e['CAT010']['I140']['ToD']],
                                     'X': [e['CAT010']['I042']['X']],
                                     'Y': [e['CAT010']['I042']['Y']],
                                     'Width': [None],
                                     'Length': [None],
                                     'Ori': [None]
                                    }
                            }
                else:
                    track = {'track': trackNb,
                            'data': {'ToD': [e['CAT010']['I140']['ToD']],
                                     'X': [None],
                                     'Y': [None],
                                     'Width': [None],
                                     'Length': [None],
                                     'Ori': [None]
                                    }
                            }
                index += 1
                trackList.append(track)
            else:
                try:
                    plotLWO = e['CAT010']['I270']
                except:
                    plotLWO = ''
                try:
                    plotXY = e['CAT010']['I042']
                except:
                    plotXY = ''
                if plotLWO and plotXY:
                    trackList[trackIndices[trackNb]]['data']['ToD'].append(e['CAT010']['I140']['ToD'])
                    trackList[trackIndices[trackNb]]['data']['X'].append(e['CAT010']['I042']['X'])
                    trackList[trackIndices[trackNb]]['data']['Y'].append(e['CAT010']['I042']['Y'])
                    trackList[trackIndices[trackNb]]['data']['Width'].append(e['CAT010']['I270']['Width'])
                    trackList[trackIndices[trackNb]]['data']['Length'].append(e['CAT010']['I270']['Length'])
                    trackList[trackIndices[trackNb]]['data']['Ori'].append(e['CAT010']['I270']['Ori'])
                elif plotXY:
                    trackList[trackIndices[trackNb]]['data']['ToD'].append(e['CAT010']['I140']['ToD'])
                    trackList[trackIndices[trackNb]]['data']['X'].append(e['CAT010']['I042']['X'])
                    trackList[trackIndices[trackNb]]['data']['Y'].append(e['CAT010']['I042']['Y'])
                    trackList[trackIndices[trackNb]]['data']['Width'].append(None)
                    trackList[trackIndices[trackNb]]['data']['Length'].append(None)
                    trackList[trackIndices[trackNb]]['data']['Ori'].append(None)
                else:
                    trackList[trackIndices[trackNb]]['data']['ToD'].append(e['CAT010']['I140']['ToD'])
                    trackList[trackIndices[trackNb]]['data']['X'].append(None)
                    trackList[trackIndices[trackNb]]['data']['Y'].append(None)
                    trackList[trackIndices[trackNb]]['data']['Width'].append(None)
                    trackList[trackIndices[trackNb]]['data']['Length'].append(None)
                    trackList[trackIndices[trackNb]]['data']['Ori'].append(None)
    return trackList, trackIndices                    


def calcStats(trackList, trackIndices):
    '''
    '''
    totalMissed = 0
    totalPlots = 0
    totalTracks = 0

    for e in trackList:
        totalTracks +=1
        # print('e[\'data\'][\'ToD\']: ', e['data']['ToD'])
        totalPlots += len(e['data']['ToD'])
        for i in range(len(e['data']['ToD'])-1):
            deltaTime = e['data']['ToD'][i+1] - e['data']['ToD'][i]
            if  deltaTime > 1.2:
                totalMissed +=1
    print('\n')
    print('Total tracks:\t', totalTracks)
    print('Total plots:\t', totalPlots, '\t\tmissed:\t', totalMissed, '\t\tpercent missed:\t', totalMissed/totalPlots*100)
    print('Prob. of Detection:\t', (totalPlots-totalMissed)/totalPlots*100, '%')


In [11]:
fileName = '20200124_4001.json'
trackList, trackIndices = readFile(fileName)

trackDF = pd.DataFrame.from_dict(trackList)
trackDF.reset_index(level=0, inplace=True)


In [40]:
trackDF[0:10]

,index,track,data
0,0,3,"{'ToD': [25155.8515625, 25156.8515625, 25157.8..."
1,1,1,"{'ToD': [25155.96875, 25156.96875, 25157.96875..."
2,2,2,"{'ToD': [25156.03125, 25157.03125, 25158.03125..."
3,3,837,"{'ToD': [25279.703125, 25280.703125, 25281.703..."
4,4,838,"{'ToD': [25301.7109375, 25302.7109375, 25303.7..."
5,5,840,"{'ToD': [25545.75, 25546.75, 25547.75, 25548.7..."
6,6,841,"{'ToD': [25562.7578125, 25563.7578125, 25564.7..."
7,7,842,"{'ToD': [25639.7578125, 25640.7734375], 'X': [..."
8,8,843,"{'ToD': [25812.25, 25813.2421875, 25814.257812..."
9,9,844,"{'ToD': [25849.875, 25850.875, 25851.875, 2585..."
